In [213]:
import pandas as pd

In [214]:
# columns = ['aid', 'aidtype', 'aidname', 'aiddesc', 'aidsrcid', 'aidsrcname',
#        'aidextid', 'aidmdate', 'cids', 'sids', 'geneids', 'aidcategories',
#        'protacxns', 'depcatg', 'pmids', 'rnai', 'ecs', 'repacxns',
#        'taxids', 'cellids', 'targettaxid', 'annotation']

# mu_agonist = pd.read_csv("bioassays/mu/mu_agonist.csv")
# mu_antagonist = pd.read_csv("bioassays/mu/mu_antagonist.csv")
# mu_partial_agonist = pd.read_csv("bioassays/mu/mu_partial_agonist.csv")
# mu_other = pd.read_csv("bioassays/mu/mu_other.csv")

# mu_agonist = mu_agonist[columns]
# mu_antagonist = mu_antagonist[columns]
# mu_partial_agonist = mu_partial_agonist[columns]
# mu_other = mu_other[columns]

# df_combined = pd.concat([mu_agonist, mu_antagonist, mu_partial_agonist, mu_other], axis=0)
# df_combined.drop_duplicates(inplace=True)
# df_combined

df_combined = pd.read_csv("PROTACXN_P35372_bioassay.csv")

In [215]:
df_combined["links"] = df_combined.aid.apply(lambda x : "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{}/CSV".format(x))
df_combined["links"]

0       https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1       https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
2       https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
3       https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
4       https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
                              ...                        
1939    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1940    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1941    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1942    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1943    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
Name: links, Length: 1944, dtype: object

In [216]:
mu_partial_agonist = pd.read_csv("bioassays/mu/mu_partial_agonist.csv")
titles = mu_partial_agonist.aiddesc.drop_duplicates().values.tolist()
# for title in titles:
#     print(title[:70])
len(titles)

36

In [217]:
mu_partial_agonist["links"] = mu_partial_agonist.aid.apply(lambda x : "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{}/CSV".format(x))
mu_partial_agonist["links"]

0     https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
1     https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
2     https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
3     https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
4     https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
                            ...                        
81    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
82    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
83    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
84    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
85    https://pubchem.ncbi.nlm.nih.gov/rest/pug/assa...
Name: links, Length: 86, dtype: object

In [417]:
idx = 36
current_title = titles[idx]
current_title

IndexError: list index out of range

In [ ]:
aid_df = df_combined[df_combined.aiddesc == current_title]
aid_links = aid_df.links.to_list()
aid_links

['https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/1668234/CSV']

In [ ]:
# aid_links.append('https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/76233/CSV')

In [ ]:
bioassays_df = []
for i, link in enumerate(aid_links):
    df = pd.read_csv(link)
    filtered_df = df[df.PUBCHEM_RESULT_TAG.str.isnumeric()]
    # filtered_df = filtered_df[  ['PUBCHEM_CID', 'Standard Type', 'Standard Relation', 'Standard Value', 'Standard Units'] ]
    try:
        filtered_df = filtered_df[  ['PUBCHEM_CID', 'Standard Type','Standard Value', 'Standard Units'] ]
        filtered_df.PUBCHEM_CID = filtered_df.PUBCHEM_CID.astype('int')
        # filtered_df.set_index('PUBCHEM_CID', inplace=True)
        filtered_df.reset_index(drop=True, inplace=True)
        filtered_df.rename(columns = {'Standard Type':'Standard_Type_' + str(i), 
                                    'Standard Value':'Standard_Value_' + str(i), 
                                    'Standard Units': 'Standard_Units_' + str(i)}, inplace = True)
        bioassays_df.append(filtered_df)
    except:
        print("idx ",i , " not working")
        pass

# bioassays_df

In [ ]:
df = bioassays_df[0]
for mined_df in bioassays_df[1:]:
    df = df.merge(mined_df, on='PUBCHEM_CID', how='outer')
df

,PUBCHEM_CID,Standard_Type_0,Standard_Value_0,Standard_Units_0
0,5497186,Ki,30.7,nM
1,156010938,Ki,NaN,NaN
2,122633395,Ki,3483,nM
3,122633436,Ki,8668,nM
4,156021880,Ki,1792,nM


In [ ]:
csv_name = current_title
print(csv_name)
df.set_index("PUBCHEM_CID", inplace=True)
df.to_csv("filtered_mu/{}_{}.csv".format(idx, csv_name[6:30]))

Title: Discovery of δ opioid receptor full agonists lacking a basic nitrogen atom and their antidepressant-like effects._||_Abstract: We have recently reported that the elaboration of the N-substituent in the δ opioid receptor (DOR) antagonist naltrindole (NTI) enabled the regulation of the DOR activities from full inverse agonists to weak partial agonists. The investigations of amide-type NTI derivatives revealed that N-phenylacetyl and N-dihydrocinnamoyl derivatives 3a and 3b were DOR full agonists. The same transformations were applied to a DOR agonist KNT-127 to provide the more potent DOR agonists 6a and 6b. Among the tested compounds, the most efficacious compound 6a showed dose-dependent antidepressant-like effects in the mouse forced swim test. The antidepressant-like effects by 6a seemed to be more potent than those of KNT-127, which is a more potent DOR agonist in in vitro assays. The amide-type compound like 6a may more fully penetrate into the central nervous system.


In [ ]:
# df = pd.read_csv("https://pubchem.ncbi.nlm.nih.gov/bioassay/395302#section=Data-Table")
# df = pd.read_csv("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/395302/CSV")
# df

In [ ]:
# filtered_df = df[df.PUBCHEM_RESULT_TAG.str.isnumeric()]
# filtered_df = filtered_df[  ['PUBCHEM_CID', 'Standard Type', 'Standard Relation' ,'Standard Value', 'Standard Units'] ]
# filtered_df.PUBCHEM_CID = filtered_df.PUBCHEM_CID.astype('int')
# filtered_df.set_index('PUBCHEM_CID', inplace=True)
# filtered_df